In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression

from feature_selectors import selection
from sklearn.preprocessing import StandardScaler

In [ ]:
gebruikers_df = pd.read_csv(r'..\..\data\gebruikersinformatie.csv', delimiter = ';')
evenement_df = pd.read_csv(r'..\..\data\evenementinformatie.csv', delimiter = ';')
aanwezigheid_df = pd.read_csv(r'..\..\data\evenementaanwezigheid.csv', delimiter = ';')
bericht_df = pd.read_csv(r'..\..\data\berichtinteracties_met_sentiment.csv', delimiter = ';')

In [ ]:
# Merge aanwezigheid_df with evenement_df
aanwezigheid_df = aanwezigheid_df.merge(evenement_df, on = "EvenementID", how = "left")

# Merge the result with gebruikers_df
aanwezigheid_df = aanwezigheid_df.merge(gebruikers_df, on = "GebruikerID", how = "left")

# Rename columns for consistency
aanwezigheid_df.rename(
    columns = {col: f"Evenement_{col}" for col in evenement_df.columns if col != "EvenementID"},
    inplace = True
)

aanwezigheid_df.rename(
    columns = {col: f"Gebruiker_{col}" for col in gebruikers_df.columns if col != "GebruikerID"},
    inplace = True
)

aanwezigheid_df

In [ ]:
# Remove nonvalid entries
aanwezigheid_df = aanwezigheid_df[~(aanwezigheid_df['Gebruiker_Leeftijd'].isna())]
aanwezigheid_df = aanwezigheid_df[~(aanwezigheid_df['Gebruiker_Lidmaatschapstype'] == 'x')]
aanwezigheid_df

In [ ]:
# Convert ID to ints
to_convert = ["EvenementID", "GebruikerID", "Evenement_OrganisatorID"]

for col in to_convert:
    aanwezigheid_df[col] = aanwezigheid_df[col].str[1:].astype(int)
    
aanwezigheid_df

In [ ]:
# Factorize
to_factorize = ["Aanwezigheidsstatus", "Evenement_EvenementType", "Gebruiker_Studierichting",
                "Gebruiker_Lidmaatschapstype"]

for col in to_factorize:
    aanwezigheid_df[col], unique_values = pd.factorize(aanwezigheid_df[col])

    print(f"\n{col} numerics:")
    for i, value in enumerate(unique_values):
        print(f"{i} -> {value}")

aanwezigheid_df["Gebruiker_Studiejaar"] = aanwezigheid_df["Gebruiker_Studiejaar"].map({
    "Eerstejaars": 1,
    "Tweedejaars": 2,
    "Derdejaars": 3,
    "Vierdejaars": 4
})

aanwezigheid_df

In [ ]:
# Convert columns to datetime datatype and extract day/month/year
date_columns = ['Evenement_EvenementDatum', 'Gebruiker_RegistratieDatum', 'Gebruiker_LaatsteLogin']

for col in date_columns:
    aanwezigheid_df[col] = pd.to_datetime(aanwezigheid_df[col], dayfirst=True)
    
for col in date_columns:
    aanwezigheid_df[f'{col}_day'] = aanwezigheid_df[col].dt.day
    aanwezigheid_df[f'{col}_month'] = aanwezigheid_df[col].dt.month
    aanwezigheid_df[f'{col}_year'] = aanwezigheid_df[col].dt.year
    
aanwezigheid_df.drop(columns = date_columns, inplace = True)

aanwezigheid_df

In [ ]:
plt.figure(figsize = (20, 8))
sns.heatmap(aanwezigheid_df.corr(), annot = True)
plt.show()

In [ ]:
# Try predicting Aanwezigheidsstatus (0 = aanwezig, 1 = afwezig)
aanwezigheid_df['Aanwezigheidsstatus'] = aanwezigheid_df['Aanwezigheidsstatus'].replace(2, 1)

all_predictors = aanwezigheid_df.columns.tolist()
outcome = 'Aanwezigheidsstatus'

all_predictors.remove(outcome)
all_predictors.remove('EvenementID')
all_predictors.remove('GebruikerID')

model = LogisticRegression()
predictors, score = selection(model, aanwezigheid_df, aanwezigheid_df[outcome], all_predictors, mode = 'forward',
                              metric = 'f1')

In [ ]:
# Test all outcomes to see which has the highest score
model = LinearRegression()
results = {}

for outcome in aanwezigheid_df.columns.tolist():
    test_predictors = aanwezigheid_df.columns.tolist()
    test_predictors.remove(outcome)

    predictors, score = selection(model, aanwezigheid_df, aanwezigheid_df[outcome], test_predictors, mode = "forward",
                                  metric = "adj_r2")

    # results[outcome] = (score, predictors)
    results[outcome] = score
    
print('\n\nOutcome\t\t\tScore')
for key, value in results.items():
    # print(f'{key} -> {value[0]}')
    print(f'{key} -> {value}')